# AdaBoost y Gradient boosting con árboles en un dataset de detección de spam

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)



(4601, 57)


## AdaBoost

Probemos el clasificador de AdaBoost

In [4]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier()

acc=ab.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


/home/yassin/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Precisión: 93.8%


**Ejercicio** Teniendo en cuenta la documentación [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) prueba mediante grid search a encontrar mejores parámetros.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

ab = AdaBoostClassifier(estimator = DecisionTreeClassifier())

# Definir los parámetros a probar
G = {
    'n_estimators': [50, 100, 150],
    'estimator__max_depth': [2, 4, 6, 8,10]
}

# Configurar el GridSearchCV
GS = GridSearchCV(ab, param_grid=G, scoring='accuracy', refit=True, cv=5,verbose =10)

# Ajustar el modelo
GS.fit(X_train, y_train)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

## GradientBoostingClassifier

Probemos el clasificador Gradient Boosting

In [6]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()

acc=gb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 95.0%


Los principales parámetros del GradientBoosting son:
* n_estimators (100)
* max_depth (3)
* max_features

**Ejercicio** prueba diferentes valores (lógicos) de estos parámetros mediante GridSearch.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
from scipy.stats import randint as sp_randint

# Definir el clasificador
gb = GradientBoostingClassifier()

# Definir los parámetros a probar
G = {
    'n_estimators': [50, 150],
    'max_depth': [3, 5],
    'max_features': ['sqrt', 'log2']
}


GS = GridSearchCV(gb, param_grid=G, scoring='accuracy', refit=True, cv=5,verbose =10)
# Ajustar el modelo
acc = GS.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START max_depth=3, max_features=sqrt, n_estimators=50.............
[CV 1/5; 1/8] END max_depth=3, max_features=sqrt, n_estimators=50;, score=0.935 total time=   0.1s
[CV 2/5; 1/8] START max_depth=3, max_features=sqrt, n_estimators=50.............
[CV 2/5; 1/8] END max_depth=3, max_features=sqrt, n_estimators=50;, score=0.928 total time=   0.1s
[CV 3/5; 1/8] START max_depth=3, max_features=sqrt, n_estimators=50.............
[CV 3/5; 1/8] END max_depth=3, max_features=sqrt, n_estimators=50;, score=0.948 total time=   0.1s
[CV 4/5; 1/8] START max_depth=3, max_features=sqrt, n_estimators=50.............
[CV 4/5; 1/8] END max_depth=3, max_features=sqrt, n_estimators=50;, score=0.936 total time=   0.1s
[CV 5/5; 1/8] START max_depth=3, max_features=sqrt, n_estimators=50.............
[CV 5/5; 1/8] END max_depth=3, max_features=sqrt, n_estimators=50;, score=0.933 total time=   0.1s
[CV 1/5; 2/8] START max_depth=3, max_fea

### Acelerar la búsqueda de parámetros: RandomizedSearch

Un problema recurrente que tenemos en estos experimentos es el coste temporal de encontrar los mejores parámetros mediante GridSearch. Una alternativa que reduce dicho coste temporal es el [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

**Ejercicio** Implementa la misma búsqueda de valores que has realizado con GridSearch pero ahora con RandomizedSearch con n_iter=10 para limitar el número de pruebas a 10.

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import numpy as np
from scipy.stats import randint as sp_randint

gb = GradientBoostingClassifier()

# Definir los parámetros a probar
param_dist = {
    'n_estimators': sp_randint(50, 150),
    'max_depth': sp_randint(3, 5),
    'max_features': ['sqrt', 'log2']
}

# Configurar el RandomizedSearchCV
RS = RandomizedSearchCV(gb, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42, verbose = 10)

# Ajustar el modelo
acc = RS.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START max_depth=3, max_features=log2, n_estimators=142...........
[CV 1/5; 1/10] END max_depth=3, max_features=log2, n_estimators=142;, score=0.944 total time=   0.2s
[CV 2/5; 1/10] START max_depth=3, max_features=log2, n_estimators=142...........
[CV 2/5; 1/10] END max_depth=3, max_features=log2, n_estimators=142;, score=0.938 total time=   0.2s
[CV 3/5; 1/10] START max_depth=3, max_features=log2, n_estimators=142...........
[CV 3/5; 1/10] END max_depth=3, max_features=log2, n_estimators=142;, score=0.959 total time=   0.2s
[CV 4/5; 1/10] START max_depth=3, max_features=log2, n_estimators=142...........
[CV 4/5; 1/10] END max_depth=3, max_features=log2, n_estimators=142;, score=0.942 total time=   0.2s
[CV 5/5; 1/10] START max_depth=3, max_features=log2, n_estimators=142...........
[CV 5/5; 1/10] END max_depth=3, max_features=log2, n_estimators=142;, score=0.954 total time=   0.2s
[CV 1/5; 2/10] START max_dept

### Mejorar la búsqueda de parámetros: BayesianOptimization

Optimización Bayesiana es una técnica avanzada de búsqueda de parámetros. Para poder emplearla hay que instalar la librería scikit-optimize. El uso de dicha técnica es similar a las ya empleadas con la diferencia que el rango de los parámetros hay que definirlos teniendo en cuenta además el tipo de los mismos: real, entero o categórico. Además los valores a probar no se listan explícitamente si no que se especifica un rango dentro del cual el algoritmo escogerá los valores.

La ejecución puede resultar más lenta pero se pueden explorar más parámetros, de hecho se especifica un intervalo en lugar de valores concretos.



In [9]:
!pip install scikit-optimize

In [10]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
BS = BayesSearchCV(GradientBoostingClassifier(), G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_depth=4, max_features=sqrt, n_estimators=52.............


/home/yassin/anaconda3/lib/python3.11/site-packages/skopt/space/space.py:110: UserWarning: Dimension [50, 150] was inferred to Integer(low=50, high=150, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(50, 150), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/home/yassin/anaconda3/lib/python3.11/site-packages/skopt/space/space.py:110: UserWarning: Dimension [3, 5] was inferred to Integer(low=3, high=5, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(3, 5), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/home/yassin/anaconda3/lib/python3.11/site-packages/skopt/space/space.py:110: UserWarning: Dimension [3, 5] was inferred to Integer(low=3, high=5, prior='uniform', transform='identity'). In upcoming versions of scik

[CV 1/5; 1/1] END max_depth=4, max_features=sqrt, n_estimators=52;, score=0.948 total time=   0.2s
[CV 2/5; 1/1] START max_depth=4, max_features=sqrt, n_estimators=52.............
[CV 2/5; 1/1] END max_depth=4, max_features=sqrt, n_estimators=52;, score=0.931 total time=   0.1s
[CV 3/5; 1/1] START max_depth=4, max_features=sqrt, n_estimators=52.............
[CV 3/5; 1/1] END max_depth=4, max_features=sqrt, n_estimators=52;, score=0.943 total time=   0.1s
[CV 4/5; 1/1] START max_depth=4, max_features=sqrt, n_estimators=52.............
[CV 4/5; 1/1] END max_depth=4, max_features=sqrt, n_estimators=52;, score=0.936 total time=   0.1s
[CV 5/5; 1/1] START max_depth=4, max_features=sqrt, n_estimators=52.............
[CV 5/5; 1/1] END max_depth=4, max_features=sqrt, n_estimators=52;, score=0.951 total time=   0.2s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_depth=5, max_features=log2, n_estimators=92.............
[CV 1/5; 1/1] END max_depth=5, max_featur

## HistGradientBoostingClassifier
Un problema del clasificador GradientBoosting implementado en sklearn es su velocidad. Es un clasificador muy lento de entrenar. Por ello sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma.

Este otro algoritmo se denomina **HistGradientBoostingClassifier**. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



**Ejercicio** Analiza en la documentación [HistGradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) los parámetros más relevantes y realiza una búsqueda de parámetros para obtener el mejor clasificador. A la hora de establecer los valores de los parámetros sería interesante fijarnos en los valores por defecto que se han empleado en el ejercicio anterior y poder explorar alrededor de dichos valores por defecto para conseguir mejores resultados.



In [11]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier()

G = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': sp_randint(50, 200),
    'max_leaf_nodes': sp_randint(20, 100),
    'max_depth': [3, 5, 7,10],
    'min_samples_leaf': sp_randint(20, 50),
    'l2_regularization': [0.0, 0.1, 0.5],
    'max_bins': sp_randint(200, 255)
}

# Configurar el RandomizedSearchCV
RS = RandomizedSearchCV(hgb, param_distributions= G, n_iter=20, refit = True, cv=5, scoring='accuracy', random_state=42, verbose=10)

# Ajustar el modelo
acc = RS.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5; 1/20] START l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40


/home/yassin/anaconda3/lib/python3.11/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


[CV 1/5; 1/20] END l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40;, score=0.914 total time=   0.3s
[CV 2/5; 1/20] START l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40
[CV 2/5; 1/20] END l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40;, score=0.906 total time=   0.2s
[CV 3/5; 1/20] START l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40
[CV 3/5; 1/20] END l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40;, score=0.928 total time=   0.3s
[CV 4/5; 1/20] START l2_regularization=0.5, learning_rate=0.01, max_bins=214, max_depth=7, max_iter=121, max_leaf_nodes=80, min_samples_leaf=40
[CV 4/5; 1/20] END l2_regularization=0.5, l